In [2]:
#necessary imports
import librosa
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import glob,os
import keras
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
# from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import matplotlib.pyplot as plt
import random

2023-08-14 11:44:25.150275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
random.seed(1)

In [4]:
# Extracting features from each .wav file
def extract_features(data, sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

In [5]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate = rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sr = sampling_rate, n_steps = pitch_factor)


In [6]:
#Emotions in the dataset
emotions={
  '01':'neutral',
  #'02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  #'07':'disgust',
  #'08':'surprised'
}

#Emotions to observe
#observed_emotions=['calm', 'happy', 'fearful', 'disgust', 'sad']


In [7]:
emotions2={
    'NEU': 'neutral',
    'HAP': 'happy',
    'SAD': 'sad',
    'ANG': 'angry',
    'FEA': 'fearful',
}

In [8]:
def get_features(path, isTraining):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    #data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    data, sample_rate = librosa.load(path, duration=2, offset=0.6, sr=8025)
    
    # without augmentation
    res1 = extract_features(data, sample_rate)
    result = np.array(res1)
    
    # Augmenting only training data and skipping augmentation for test data
    if isTraining:
        
        # data with noise
        noise_data = noise(data)
        res2 = extract_features(noise_data, sample_rate)
        result = np.vstack((result, res2)) # stacking vertically

        # data with stretching 
        stretched_data = stretch(data)
        res3 = extract_features(stretched_data, sample_rate)
        result = np.vstack((result, res3)) # stacking vertically
        
        # data with pitch offset
        data_pitch = pitch(data, sample_rate)
        res4 = extract_features(data_pitch, sample_rate)
        result = np.vstack((result, res4)) # stacking vertically
    
    return result

# New Data
## Crema-D dataset

In [18]:
Creama = "data2/"
creama_directory_list = os.listdir(Creama)

file_emotion = []
file_path = []
for file in glob.glob(f"{Creama}*"):
    part = file.split('.')[0]
    part = part.split('_')
    if part[2] not in emotions2.keys():
        continue
    file_emotion.append(part[2])
    file_path.append(file)

# dataframe for emotion of files
emotion2_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files
path2_df = pd.DataFrame(file_path, columns=['Path'])
Creama_df = pd.concat([emotion2_df, path2_df], axis=1)

# Mapping integers to corresponding emotions
Creama_df = Creama_df.replace({'Emotions': emotions2})
print(Creama_df["Emotions"].value_counts())


angry      1271
sad        1271
fearful    1271
happy      1271
neutral    1087
Name: Emotions, dtype: int64


In [19]:
# Creating Dataframes
Ravdess = "data/"
ravdess_directory_list = os.listdir(Ravdess)
file_emotion = []
file_path = []

for folder in glob.glob(f"{Ravdess}Actor_*"):
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(folder)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        if part[2] not in emotions.keys():
            continue
        file_emotion.append(int(part[2]))
        file_path.append(folder + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])


# dataframe for path of files
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# Mapping integers to corresponding emotions
Ravdess_df.Emotions.replace({1:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fearful'}, inplace=True)

Ravdess_df['Emotions'].value_counts()

angry      192
fearful    192
sad        192
happy      192
neutral     96
Name: Emotions, dtype: int64

In [20]:
data = pd.concat([Creama_df, Ravdess_df], axis=0)

X = data["Path"]
Y = data["Emotions"]

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, shuffle=True)

In [22]:
Y_train.value_counts(), Y_test.value_counts()

(sad        1129
 happy      1099
 fearful    1096
 angry      1071
 neutral     881
 Name: Emotions, dtype: int64,
 angry      392
 fearful    367
 happy      364
 sad        334
 neutral    302
 Name: Emotions, dtype: int64)

In [23]:
print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_test))
print(np.shape(Y_test))

(5276,)
(5276,)
(1759,)
(1759,)


In [24]:
X, Y = [], []
for path, emotion in zip(X_train, Y_train):
    feature = get_features(path, True)
    for ele in feature:
        X.append(ele)
        # appending emotion 4 times as we have applied 4 augmentation techniques on each audio file
        Y.append(emotion)
x_train = X
y_train = Y

KeyboardInterrupt: 

In [ ]:
# Saving features in a csv file to avoid redundant feature extraction
Features = pd.DataFrame(x_train)
Features['labels'] = y_train
Features.to_csv('features.csv', index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.132812,0.786461,0.788402,0.502173,0.456299,0.488599,0.458573,0.497425,0.589869,0.639345,...,0.106477,0.037218,0.030323,0.022959,0.034632,0.018931,0.007288,0.001811,0.000043,angry
1,0.197499,0.794354,0.684380,0.456011,0.470637,0.497837,0.486151,0.527306,0.641404,0.627995,...,0.105259,0.037137,0.030578,0.023704,0.035130,0.019389,0.007926,0.002445,0.000779,angry
2,0.155408,0.906215,0.619200,0.418835,0.486907,0.462776,0.463361,0.517800,0.655547,0.588477,...,0.066172,0.022858,0.017371,0.012629,0.020867,0.012879,0.005110,0.001129,0.000024,angry
3,0.161833,0.527145,0.869874,0.795607,0.476046,0.447058,0.478203,0.410140,0.459286,0.580427,...,0.083353,0.028837,0.070650,0.033715,0.023035,0.016411,0.013157,0.004276,0.000025,angry
4,0.123682,0.609155,0.439890,0.468290,0.535586,0.620048,0.572363,0.580419,0.533955,0.571267,...,0.033259,0.025641,0.091899,0.020896,0.011710,0.024206,0.016897,0.003146,0.000130,angry


In [ ]:
x_train = Features.iloc[: ,:-1].values
y_train = Features['labels'].values
print(y_train)

['angry' 'angry' 'angry' ... 'fearful' 'fearful' 'fearful']


In [ ]:
encoder = sklearn.preprocessing.OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
print(y_train)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [ ]:
X, Y = [], []
for path, emotion in zip(X_test, Y_test):
    feature = get_features(path, False)   
    X.append(feature)
    Y.append(emotion)

print(np.shape(X))
print(np.shape(Y))
x_test = np.array(X)
y_test = np.array(Y)


(1759, 162)
(1759,)


In [ ]:
Y_test.head(5)

4003      happy
474         sad
3927        sad
4694    fearful
1237    fearful
Name: Emotions, dtype: object

In [ ]:
X_test.head(5)

4003                 data2/1048_IEO_HAP_MD.wav
474     data/Actor_09/03-01-04-02-01-02-09.wav
3927                 data2/1038_TSI_SAD_XX.wav
4694                 data2/1024_TIE_FEA_XX.wav
1237                 data2/1044_TSI_FEA_XX.wav
Name: Path, dtype: object

In [ ]:
encoder = sklearn.preprocessing.OneHotEncoder()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()

In [ ]:
# Normalization
scaler = sklearn.preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((21104, 162), (21104, 6), (1759, 162), (1759, 6))

In [ ]:
model1=MLPClassifier(alpha=0.01, batch_size=64, epsilon=1e-08, hidden_layer_sizes=(500,), learning_rate='adaptive', max_iter=500)
model1.fit(x_train,y_train)
score = model1.score(x_test, y_test)
print("Accuracy:", score)

Accuracy: 0.4019329164297897


In [ ]:
# Reshaping train and test data for new model
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((21104, 162, 1), (21104, 6), (1759, 162, 1), (1759, 6))